# Agregaciones

Las agregaciones se refieren a la operación de calcular estadísticas resumidas sobre un conjunto de datos en función de una o varias columnas de agrupación. Las estadísticas que se pueden calcular incluyen la suma, el promedio, el máximo, el mínimo, la desviación estándar, la mediana, el número de valores distintos, entre otras.

Las agregaciones se realizan mediante el método groupBy() en un objeto DataFrame, que agrupa los datos en función de una o varias columnas. A continuación, se aplican una o varias funciones de agregación a este objeto GroupedData para calcular las estadísticas resumidas.

Sin embargo, no todas las funciones que calculan estadísticas resumidas en PySpark requieren el uso de groupBy(). Por ejemplo, la función count() cuenta el número de filas en un DataFrame sin la necesidad de agrupar por ninguna columna, y se puede aplicar directamente sobre un DataFrame. Por lo tanto, aunque count() es una función de agregación, no requiere el uso de groupBy().

En resumen, las agregaciones en PySpark se refieren al cálculo de estadísticas resumidas sobre un conjunto de datos en función de una o varias columnas de agrupación. El uso de la función groupBy() es una forma común de realizar las agregaciones, pero no es necesario en todos los casos.

In [80]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, desc, sum, sum_distinct, min, max, avg, count, countDistinct, approx_count_distinct, broadcast

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [2]:
df = spark.read.parquet(
    r"../../data/vuelos.parquet",
)

df.printSchema()

NameError: name 'spark' is not defined

## Funciones count, countDistinct y approx_count_distinct

* **count()**: devuelve el número total de filas en un grupo.
* **countDistinct()**: devuelve el número de valores distintos de una columna en un grupo.
* **approx_count_distinct()**: devuelve el número aproximado de valores distintos de una columna en un grupo.

In [5]:
df = spark.read.parquet(
    r"../../data/dataframe.parquet",
)

df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- color: string (nullable = true)
 |-- cantidad: long (nullable = true)



In [6]:
df.show()

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  null| null|    1700|
|  null| rojo|    1300|
|  Juan| rojo|    1500|
+------+-----+--------+



In [9]:
# count
df.select(
    count("nombre").alias("nombre_count"),
    count("color").alias("color_count"),
    count("*").alias("general_count"),  # cuenta todas las filas
).show()

+------------+-----------+-------------+
|nombre_count|color_count|general_count|
+------------+-----------+-------------+
|           2|          3|            4|
+------------+-----------+-------------+



In [10]:
# countDistinct
df.select(
    countDistinct("nombre").alias("nombre_count"),
    countDistinct("color").alias("color_count"),
    countDistinct("*").alias("general_count"),  # cuenta todas las filas
).show()

+------------+-----------+-------------+
|nombre_count|color_count|general_count|
+------------+-----------+-------------+
|           2|          2|            2|
+------------+-----------+-------------+



In [4]:
# approx_count_distinct

df_vuelos = spark.read.parquet(
    r"../../data/vuelos.parquet",
)

df_vuelos.select(
    approx_count_distinct("AIRLINE").alias("aerolinea_count"),
    # Es una aproximación del número de valores distintos de la columna AIRLINE en el DataFrame
).show()

+---------------+
|aerolinea_count|
+---------------+
|             13|
+---------------+



In [17]:
# En realidad tiene 14 valores distintos, pero approx_count_distinct devuelve 13
df_vuelos.select(
    countDistinct("AIRLINE").alias("aerolinea_count"),
    # Es una aproximación del número de valores distintos de la columna AIRLINE en el DataFrame
).show()

+---------------+
|aerolinea_count|
+---------------+
|             14|
+---------------+



## Funciones min y max

* **min()**: devuelve el valor mínimo de una columna en un grupo.
* **max()**: devuelve el valor máximo de una columna en un grupo.

In [6]:
# min
df_vuelos.select(
    min("AIR_TIME").alias("min_air_time"),
    max("AIR_TIME").alias("max_air_time"),
).show()

+------------+------------+
|min_air_time|max_air_time|
+------------+------------+
|           7|         690|
+------------+------------+



## Funciones sum, sumDistinct y avg

* **sum()**: devuelve la suma de los valores de una columna en un grupo.
* **sumDistinct()**: devuelve la suma de los valores distintos de una columna en un grupo.
* **avg()**: devuelve el promedio de los valores de una columna en un grupo.

In [9]:
df_vuelos.select(
    sum("DISTANCE").alias("sum_distance"),
    sum_distinct("DISTANCE").alias("sum_distinct_distance"),
    avg("DISTANCE").alias("avg_distance"),
).show()

+------------+---------------------+-----------------+
|sum_distance|sum_distinct_distance|     avg_distance|
+------------+---------------------+-----------------+
|  4785357409|              1442300|822.3564947305235|
+------------+---------------------+-----------------+



AttributeError: 'GroupedData' object has no attribute 'show'

## Agregación con agrupación

Realizar agregación con agrupación es un proceso de dos pasos:

1. Realizar la agrupación mediante la transformación groupBy().
2. Aplicar una o varias funciones de agregación al objeto GroupedData resultante.

In [10]:
df_vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [14]:
# Agrupación por aeropuerto de origen
df_vuelos.groupBy("ORIGIN_AIRPORT").count().orderBy(desc("count")).show()

+--------------+------+
|ORIGIN_AIRPORT| count|
+--------------+------+
|           ATL|346836|
|           ORD|285884|
|           DFW|239551|
|           DEN|196055|
|           LAX|194673|
|           SFO|148008|
|           PHX|146815|
|           IAH|146622|
|           LAS|133181|
|           MSP|112117|
|           MCO|110982|
|           SEA|110899|
|           DTW|108500|
|           BOS|107847|
|           EWR|101772|
|           CLT|100324|
|           LGA| 99605|
|           SLC| 97210|
|           JFK| 93811|
|           BWI| 86079|
+--------------+------+
only showing top 20 rows



In [17]:
(
    df_vuelos.
    groupBy(
        "ORIGIN_AIRPORT",
        "DESTINATION_AIRPORT"
    )
    .count()
    .orderBy(
        desc("count")
    )
    .show()
)

+--------------+-------------------+-----+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+-------------------+-----+
|           SFO|                LAX|13744|
|           LAX|                SFO|13457|
|           JFK|                LAX|12016|
|           LAX|                JFK|12015|
|           LAS|                LAX| 9715|
|           LGA|                ORD| 9639|
|           LAX|                LAS| 9594|
|           ORD|                LGA| 9575|
|           SFO|                JFK| 8440|
|           JFK|                SFO| 8437|
|           OGG|                HNL| 8313|
|           HNL|                OGG| 8282|
|           LAX|                ORD| 8256|
|           ATL|                LGA| 8234|
|           LGA|                ATL| 8215|
|           ATL|                MCO| 8202|
|           MCO|                ATL| 8202|
|           SFO|                LAS| 7995|
|           ORD|                LAX| 7941|
|           LAS|                SFO| 7870|
+----------

In [24]:
df_vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [25]:
# Múltiples agregaciones por grupo
(
    df_vuelos.
    groupBy(
        "ORIGIN_AIRPORT",
    )
    .agg(
        count("*").alias("count"),
        avg("DISTANCE").alias("avg_distance"),
        min("DISTANCE").alias("min_distance"),
        max("DISTANCE").alias("max_distance"),
    )
    .orderBy(
        desc("avg_distance"),
    )
    .show()
)

+--------------+------+------------------+------------+------------+
|ORIGIN_AIRPORT| count|      avg_distance|min_distance|max_distance|
+--------------+------+------------------+------------+------------+
|         12016|    31|            3801.0|        3801|        3801|
|           GUM|   334|            3801.0|        3801|        3801|
|           PPG|   107|            2599.0|        2599|        2599|
|         14222|     9|            2599.0|        2599|        2599|
|         12478|  8303|1476.6809586896302|         173|        4983|
|           JFK| 93811|1441.8611463474433|         173|        4983|
|         14254|    62|            1398.0|        1179|        1617|
|           ANC| 16005|1397.8466104342392|         160|        3417|
|           SJU| 24656|1385.0269305645684|          68|        2404|
|           PSE|   749|1373.7316421895862|        1179|        1617|
|         14843|  1868| 1368.460920770878|          68|        2165|
|         12264|  2996|1365.320093

In [31]:
(
    df_vuelos
    .groupby(
        "MONTH",
    )
    .agg(
        count("*").alias("count"),
        avg("ARRIVAL_DELAY").alias("avg_arrival_delay"),
    )
    .orderBy(
        "MONTH",
    )
    .show()
)

+-----+------+-------------------+
|MONTH| count|  avg_arrival_delay|
+-----+------+-------------------+
|    1|469968|  5.813582983416227|
|    2|429191|  8.320500020850556|
|    3|504312|   4.92067265685641|
|    4|485151| 3.1631900611579318|
|    5|496993|  4.485018615679651|
|    6|503897|  9.601590351569554|
|    7|520718| 6.4317747052785466|
|    8|510536|  4.607372469025074|
|    9|464946|-0.7725709883956179|
|   10|486165| -0.780389663641748|
|   11|467972|  1.100783576682592|
|   12|479230|  6.092902747824754|
+-----+------+-------------------+



## Agregación con Pivote

La función pivot en PySpark se utiliza para realizar una operación de pivote en un DataFrame. El pivote es una operación que permite reorganizar los datos en un DataFrame, convirtiendo los valores de una columna en nuevas columnas.

En la operación de pivote, debes especificar tres elementos principales:

1. La columna que se utilizará como referencia para crear las nuevas columnas.
2. Los valores únicos de esa columna que se convertirán en las nuevas columnas.
3. La columna cuyos valores se agregarán en las nuevas columnas creadas.

In [33]:
df_estudiantes = spark.read.parquet(
    r"../../data/estudiantes.parquet",
)
df_estudiantes.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- peso: long (nullable = true)
 |-- graduacion: long (nullable = true)



In [34]:
df_estudiantes.show()

+------+----+----+----------+
|nombre|sexo|peso|graduacion|
+------+----+----+----------+
|  Jose|   M|  80|      2000|
| Hilda|   F|  50|      2000|
|  Juan|   M|  75|      2000|
| Pedro|   M|  76|      2001|
|Katia+|   F|  65|      2001|
+------+----+----+----------+



In [38]:
# avg, min, max de peso por año de graduación y sexo

(
    df_estudiantes
    .groupBy(
        "graduacion",
    )
    .pivot(
        "sexo",
    )
    .agg(
        avg("peso").alias("avg_peso"),
        min("peso").alias("min_peso"),
        max("peso").alias("max_peso"),
    )
    .orderBy(
        "graduacion",
    )
    .show()
)

+----------+----------+----------+----------+
|graduacion|M_avg_peso|M_min_peso|M_max_peso|
+----------+----------+----------+----------+
|      2000|      77.5|        75|        80|
|      2001|      76.0|        76|        76|
+----------+----------+----------+----------+



In [39]:
# Pivot con ciertos valores

(
    df_estudiantes
    .groupBy(
        "graduacion",
    )
    .pivot(
        pivot_col="sexo",
        values=["M"],  # solo se toman en cuenta los valores M de la columna sexo
    )
    .agg(
        avg("peso").alias("avg_peso"),
        min("peso").alias("min_peso"),
        max("peso").alias("max_peso"),
    )
    .orderBy(
        "graduacion",
    )
    .show()
)

+----------+----------+----------+----------+
|graduacion|M_avg_peso|M_min_peso|M_max_peso|
+----------+----------+----------+----------+
|      2000|      77.5|        75|        80|
|      2001|      76.0|        76|        76|
+----------+----------+----------+----------+



# Joins

Un join es una operación que combina dos DataFrames basándose en una columna común. La columna común se utiliza para comparar los valores en ambas tablas y unir las filas que coincidan según un criterio específico.

Existen varios tipos de join en PySpark, que determinan cómo se realizan las combinaciones de filas entre los DataFrames. A continuación, te explico los tipos de join más comunes:

* **Inner Join**: Es el tipo de join más común. Solo incluye las filas en el resultado final que tienen una coincidencia en ambas tablas en la columna especificada. En otras palabras, se seleccionan solo las filas que tienen un valor común en la columna utilizada para el join.

* **Left Join**: Incluye todas las filas del DataFrame izquierdo (el primer DataFrame en la operación join) y las filas coincidentes del DataFrame derecho. Si no hay coincidencias en el DataFrame derecho, se rellenan con valores nulos.

* **Right Join**: Es similar al left join, pero incluye todas las filas del DataFrame derecho y las filas coincidentes del DataFrame izquierdo. Si no hay coincidencias en el DataFrame izquierdo, se rellenan con valores nulos.

* **Full Outer Join**: Incluye todas las filas de ambos DataFrames en el resultado final, y rellena con valores nulos en las columnas correspondientes si no hay coincidencias.

* **Left Semi Join**: Devuelve las filas del DataFrame izquierdo que tienen coincidencias en el DataFrame derecho, sin incluir realmente las columnas del DataFrame derecho en el resultado.

* **Left Anti Join**: Devuelve las filas del DataFrame izquierdo que no tienen coincidencias en el DataFrame derecho.

* **Cross Join**: Devuelve el producto cartesiano de ambos DataFrames. Es decir, combina cada fila del DataFrame izquierdo con cada fila del DataFrame derecho.

In [40]:
df_empleados = spark.read.parquet(
    r"../../data/empleados.parquet",
)

df_departamentos = spark.read.parquet(
    r"../../data/departamentos.parquet",
)

In [41]:
df_empleados.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [42]:
df_departamentos.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



In [43]:
df_empleados.show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [44]:
df_departamentos.show()

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 31|     letras|
| 33|    derecho|
| 34| matemática|
| 35|informática|
+---+-----------+



In [49]:
# Inner Join
(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="inner",
    )
    .show()
)
# O
(
    df_empleados
    .join(
        df_departamentos,
    )
    .where(
        df_empleados.num_dpto == df_departamentos.id,
    )
    .show()
)

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [50]:
# Left Outer Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="left",
    )
    .show()
)

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Pedro|       0|null|       null|
| Laura|      34|  34| matemática|
|Sandro|      31|  31|     letras|
+------+--------+----+-----------+



In [51]:
# Right Outer Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="right",
    )
    .show()
)

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|Sandro|      31| 31|     letras|
| Katia|      33| 33|    derecho|
|  Luis|      33| 33|    derecho|
| Laura|      34| 34| matemática|
|  Raul|      34| 34| matemática|
|  null|    null| 35|informática|
+------+--------+---+-----------+



In [52]:
# Full Outer Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="full",
    )
    .show()
)

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
| Pedro|       0|null|       null|
|Sandro|      31|  31|     letras|
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Laura|      34|  34| matemática|
|  null|    null|  35|informática|
+------+--------+----+-----------+



In [53]:
# Left Semi Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="left_semi",
    )
    .show()
)

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [54]:
# Left Anti Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="left_anti",
    )
    .show()
)

+------+--------+
|nombre|num_dpto|
+------+--------+
| Pedro|       0|
+------+--------+



In [56]:
# Cross Join

(
    df_empleados
    .crossJoin(
        df_departamentos,
    )
    .show()
)

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 31|     letras|
|  Luis|      33| 33|    derecho|
|  Luis|      33| 34| matemática|
|  Luis|      33| 35|informática|
| Katia|      33| 31|     letras|
| Katia|      33| 33|    derecho|
| Katia|      33| 34| matemática|
| Katia|      33| 35|informática|
|  Raul|      34| 31|     letras|
|  Raul|      34| 33|    derecho|
|  Raul|      34| 34| matemática|
|  Raul|      34| 35|informática|
| Pedro|       0| 31|     letras|
| Pedro|       0| 33|    derecho|
| Pedro|       0| 34| matemática|
| Pedro|       0| 35|informática|
| Laura|      34| 31|     letras|
| Laura|      34| 33|    derecho|
| Laura|      34| 34| matemática|
| Laura|      34| 35|informática|
+------+--------+---+-----------+
only showing top 20 rows



# Manejo de nombres de columnas duplicados

In [59]:
df_depa = df_departamentos.withColumn("num_dpto", col("id"))

df_depa.printSchema()

df_empleados.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [62]:
# Join error

(
    df_empleados
    .join(
        df_depa,
        on=col("num_dpto") == col("num_dpto"),
        how="inner",
    )
    .show()
)

AnalysisException: Reference 'num_dpto' is ambiguous, could be: num_dpto, num_dpto.

In [68]:
# Especificando el DF

df_result = (
    df_empleados
    .join(
        df_depa,
        on=df_empleados.num_dpto == df_depa.num_dpto,  # o df_empleados["num_dpto"] == df_depa["num_dpto"]
        how="inner",
    )
)

df_result.show()

+------+--------+---+-----------+--------+
|nombre|num_dpto| id|nombre_dpto|num_dpto|
+------+--------+---+-----------+--------+
|Sandro|      31| 31|     letras|      31|
| Katia|      33| 33|    derecho|      33|
|  Luis|      33| 33|    derecho|      33|
| Laura|      34| 34| matemática|      34|
|  Raul|      34| 34| matemática|      34|
+------+--------+---+-----------+--------+



In [69]:
# Select error

df_result.select("num_dpto").show()

AnalysisException: Reference 'num_dpto' is ambiguous, could be: num_dpto, num_dpto.

In [71]:
# Select con DF especificado

df_result.select(df_empleados.num_dpto).show()

+--------+
|num_dpto|
+--------+
|      33|
|      33|
|      34|
|      34|
|      31|
+--------+



In [75]:
# Utilizando una columna de Union

dummy_df = df_empleados.join(
    df_depa,
    on="num_dpto",
    how="inner",
)

In [76]:
dummy_df.printSchema()  # num_dpto es la columna de union, solo aparece una vez

root
 |-- num_dpto: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



# Shuffle Hash Join y Broadcast Hash Join

El shuffle y el broadcast hash join son técnicas utilizadas para optimizar las operaciones de unión (join) entre DataFrames o RDDs distribuidos. A continuación, se explica cada uno:

* **Shuffle**: Shuffle es una operación costosa que implica la redistribución de datos entre los nodos del clúster. En el contexto de un join, implica la reorganización de los datos para asegurar que las claves de unión se encuentren en el mismo nodo. Esto es necesario para que los registros con las mismas claves se combinen correctamente.

Durante un shuffle, los datos se particionan y se envían a diferentes nodos según un criterio de particionamiento, como una función de hash de las claves de unión. Luego, los nodos realizan la combinación de los registros que tienen las mismas claves. El shuffle puede ser intensivo en términos de tiempo y recursos, ya que requiere transferir y reorganizar grandes volúmenes de datos entre los nodos.

* **Broadcast Hash Join**: En contraste con el shuffle, el broadcast hash join es una estrategia de optimización que evita la necesidad de realizar un shuffle. En este caso, una de las tablas involucradas en el join (la tabla más pequeña) se "broadcastea" o se distribuye a todos los nodos del clúster. Esto significa que cada nodo tiene una copia local de la tabla pequeña.

Al tener la tabla pequeña en cada nodo, el join se realiza localmente en cada nodo sin necesidad de reorganizar datos o realizar transferencias entre nodos. Este enfoque es eficiente cuando una de las tablas es lo suficientemente pequeña para caber en la memoria de cada nodo.

Sin embargo, es importante tener en cuenta que el broadcast hash join solo es beneficioso cuando la tabla pequeña es lo suficientemente pequeña para caber en la memoria de cada nodo. Si la tabla pequeña es demasiado grande, puede llevar a problemas de rendimiento debido al consumo excesivo de memoria en cada nodo.

In [77]:
df_empleados = spark.read.parquet(
    r"../../data/empleados.parquet",
)

df_departamentos = spark.read.parquet(
    r"../../data/departamentos.parquet",
)

df_empleados.printSchema()

df_departamentos.printSchema()

df_empleados.show()

df_departamentos.show()

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 31|     letras|
| 33|    derecho|
| 34| matemática|
| 35|informática|
+---+-----------+



In [78]:
# Shuffle Hash Join

(
    df_empleados
    .join(
        df_departamentos,
        on=df_empleados.num_dpto == df_departamentos.id,
        how="inner",
    )
    .explain()
)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#6116L], [id#6119L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#6116L)
   :  +- FileScan parquet [nombre#6115,num_dpto#6116L] Batched: true, DataFilters: [isnotnull(num_dpto#6116L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/B:/OneDrive/Coding/Python/Courses/PySpark/ingenieria-datos-pytho..., PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=3196]
      +- Filter isnotnull(id#6119L)
         +- FileScan parquet [id#6119L,nombre_dpto#6120] Batched: true, DataFilters: [isnotnull(id#6119L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/B:/OneDrive/Coding/Python/Courses/PySpark/ingenieria-datos-pytho..., PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:strin

In [81]:
# Broadcast Hash Join

(
    df_empleados
    .join(
        broadcast(df_departamentos),
        on=df_empleados.num_dpto == df_departamentos.id,
        how="inner",
    )
    .explain()
)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#6116L], [id#6119L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#6116L)
   :  +- FileScan parquet [nombre#6115,num_dpto#6116L] Batched: true, DataFilters: [isnotnull(num_dpto#6116L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/B:/OneDrive/Coding/Python/Courses/PySpark/ingenieria-datos-pytho..., PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=3219]
      +- Filter isnotnull(id#6119L)
         +- FileScan parquet [id#6119L,nombre_dpto#6120] Batched: true, DataFilters: [isnotnull(id#6119L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/B:/OneDrive/Coding/Python/Courses/PySpark/ingenieria-datos-pytho..., PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:strin